In [7]:
# common
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

from datetime import datetime, timedelta
from io import BytesIO
from pathlib import Path

# network & protocol
import requests
from requests.exceptions import BaseHTTPError

# computational
import numpy as np
import pandas as pd

In [8]:
# stations with no discharge to calculate rating curves
# Kg. Chhnang, Kompong Luong,
stations = [
    {
        'name': 'Neak_Luong',
        'discharge': '661d84572a544af4b7f28d9dc685d4a8',
        'stage': '0c55bc1fb44e4caf881226bbf8a6e62a',
    },
    {
        'name': 'Stung_Treng',
        'discharge': {
            'calculated': '2066a06f57764afca57a6be0f3eff8a7',
            'measured': 'dcd4b92a9d9341859f5f1658836166f1',
        },
        'stage': {
            'telemetry': '0a723f8802e44a6bb12c08dcf1816a83',
            'manual': '000e84564d1b425ebcc2c2292ed14061'
        },
    },
    {
        'name': 'Chaktomuk',
        'discharge': '983ed55df7e04410b23fc6d72eb8f235',
        'stage': '7f84c4d851a543d1a5eccc53947e0d3c',
    },
    {
        'name': 'Kompong_Kdei',
        'discharge': '2d363b26ee524761b5d2da2a0ef59613',
        'stage': '75ba98758e7c4eb3b105f733db150307',
    },
    {
        'name': 'Prek_Kdam',
        'discharge': {
            'calculated': '5579eb91df394552bac8c939724ba6cc',
            'measured': '6615982ab17b4a38a5269e5ba4ef8a4f',
        },
        'stage': {
            'telemetry': '5a7037f272c745d5bcafcc16a5c2f89c',
            'manual': '81e259eb15f64135834fe9e388ee7fbb'
        },
    },
    {
        'name': 'Kg._Thmar',
        'discharge': '2575ba232eb1406a8ce460de9fd5849a',
        'stage': '549c7f63d6dc4392b233b2049bd23d11',
    },
    {
        'name': 'Koh_Khel',
        'discharge': 'e2b360019f3f4c438766ab8ed04286aa',
        'stage': '64758ae8f24a46759d962ab9a2dea27a',
    },
    {
        'name': 'Lumphat',
        'discharge': 'f17adb5f893a4f40924f698978906b87',
        'stage': {
            'telemetry': '6955b9585cc94524bcff184034e4afd3',
            'manual': '93337ffe7838467289bf5b55401ee6f8'
        },
    },
    {
        'name': 'Kompong_Cham',
        'discharge': '30f15c16035f4bcfbac36fc545fb132a',
        'stage': 'b5da800e9c8e4346a3b8d9d6daa73e0a',
    },
    {
        'name': 'Phnom_Penh_Port',
        'discharge': 'e63fa05dafea4873bd44574125a97495',
        'stage': 'cd94408bcf944bc88033cf05675034fd',
    },
    {
        'name': 'Voeun_Sai',
        'discharge': 'eacd08c3a27e4fc5a3aa1af35792f6e3',
        'stage': {
            'telemetry': 'e821b15baedd4af2a4ecde624b3bc976',
            'manual': 'd177b0f2dae547ceb9bc7c23ef2970ae'
        },
    },
    {
        'name': 'Kratie',
        'discharge': {
            'calculated': '17c0ffe74e224b50a9e3142fc404262b',
            'measured': 'ecc50c5540564a27bb9f7f147027bd9f',
        },
        'stage': {
            'telemetry': '39b735a5de5c440cb0bc789fb9fdd544',
            'manual': 'fca7bee14dcf43a59263158f18a88509'
        },
    },
    {
        'name': 'Kompong_Chen',
        'discharge': '0ef0b44545814f8fad8905bae66a4f11',
        'stage': '53359074bfcf4961866daac9fbee8292',
    },
    {
        'name': 'Kompong_Thom',
        'discharge': '6b283ac2d44e476eba80325fba62b0b7',
        'stage': {
            'telemetry': 'aaebd95890ca49b9b5b5a82699159d6d',
            'manual': 'dae00e365f134b4bbfa6bf4162eb815c'
        },
    },
    {
        'name': 'Siempang',
        'discharge': '3f588063c8bd4f06a477251701ec6ab0',
        'stage': {
            'telemetry': '5794c0c5c5c045639e0efa1540466828',
            'manual': '6a49fa1f6cb247d39b9e6c405205bfa3'
        },
    },
    {
        'name': 'Battambang',
        'discharge': '09cc6bff5c604eafb79eaf233ab63bf1',
        'stage': {
            'telemetry': '8456239a6baf4221a744886c8cb6efd3',
            'manual': 'cba718574b1f4244a4ae2ef58806732d'
        },
    },
    {
        'name': 'Sisophon',
        'discharge': '298b8492ba45480888b8043d2c9fa17f',
        'stage': {
            'telemetry': 'f08efea6bd684203ab33c5a316e2459a',
            'manual': '2a8a001888a84456916ff7a89be300e7'
        },
    },
]

In [9]:
Path(f'{BASE_DIR}/pre_rating_curves').mkdir(parents=True, exist_ok=True)

for station in stations:
    name = station['name']
    print('-------------------------------------------')
    print(f'station: {name}')

    _discharge = station['discharge']
    discharge = [_discharge] if isinstance(_discharge, str) else [_discharge['calculated'], _discharge['measured']]
    _stage = station['stage']
    stage = [_stage] if isinstance(_stage, str) else [_stage['telemetry'], _stage['manual']]

    cols = ['datetime', 'discharge', 'calculated']
    df_discharge = pd.DataFrame(columns = cols)

    for _id in discharge:
        _df = pd.read_csv(f'{BASE_DIR}/mrc_download/{_id}.csv', index_col='datetime')
        
        if not _df.empty:
            _df = _df.rename(columns={'value': 'discharge'})

            if not df_discharge.empty:
                _df['temp'] = _df['discharge']
                            
                df_discharge = pd.concat([df_discharge, _df['temp']], axis=1)
                date_overlap = df_discharge['temp'].notnull()

                df_discharge['calculated'].where(~date_overlap, 'N', inplace=True)
                df_discharge['discharge'].where(~date_overlap, df_discharge['temp'], inplace=True)
                df_discharge.drop(['temp'], axis=1, inplace=True)
            else:
                _df['calculated'] = 'Y'
                # comes first time for calculated
                df_discharge = _df

    df_discharge.sort_values(by=['datetime'], ascending=True, inplace=True)
    Path(f'{BASE_DIR}/pre_rating_curves/{name}').mkdir(parents=True, exist_ok=True)
    df_discharge.to_csv(f'{BASE_DIR}/pre_rating_curves/{name}/discharge.csv', encoding='utf-8', index=True)
    

    cols = ['datetime', 'stage', 'manual']
    df_stage = pd.DataFrame(columns = cols)

    for _id in stage:
        _df = pd.read_csv(f'{BASE_DIR}/mrc_download/{_id}.csv', index_col='datetime')

        if not _df.empty:
            _df = _df.rename(columns={'value': 'stage'})

            if not df_stage.empty:
                _df['temp'] = _df['stage']

                df_stage = pd.concat([df_stage, _df['temp']], axis=1)
                date_overlap = df_stage['temp'].notnull()

                df_stage['manual'].where(~date_overlap, 'Y', inplace=True)
                df_stage['stage'].where(~date_overlap, df_stage['temp'], inplace=True)
                df_stage.drop(['temp'], axis=1, inplace=True)
            else:
                if len(stage) > 1:
                    _df['manual'] = 'N'
                else:
                    _df['manual'] = 'Y'
                # comes first time for telemetry
                df_stage = _df

    df_stage.sort_values(by=['datetime'], ascending=True, inplace=True)
    Path(f'{BASE_DIR}/pre_rating_curves/{name}').mkdir(parents=True, exist_ok=True)
    df_stage.to_csv(f'{BASE_DIR}/pre_rating_curves/{name}/stage.csv', encoding='utf-8', index=True)
    

-------------------------------------------
station: Neak_Luong
-------------------------------------------
station: Stung_Treng
-------------------------------------------
station: Chaktomuk
-------------------------------------------
station: Kompong_Kdei
-------------------------------------------
station: Prek_Kdam
-------------------------------------------
station: Kg._Thmar
-------------------------------------------
station: Koh_Khel
-------------------------------------------
station: Lumphat
-------------------------------------------
station: Kompong_Cham
-------------------------------------------
station: Phnom_Penh_Port
-------------------------------------------
station: Voeun_Sai
-------------------------------------------
station: Kratie
-------------------------------------------
station: Kompong_Chen
-------------------------------------------
station: Kompong_Thom
-------------------------------------------
station: Siempang
-----------------------------------------